# Axiom $T$

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- 1. CORE MLNN OPERATORS (Differentiable Kripke Semantics) ---

def softmin(x, tau=0.1):
    """Equation 1 & Section 3.2.1: Sound lower bound on necessity quantification."""
    return -tau * torch.log(torch.sum(torch.exp(-x / tau), dim=-1) + 1e-8)

def mlnn_box(A, L_phi):
    """
    Implements Necessity Neuron (Section 3.2.1).
    L_box = softmin( (1 - A_ij) + L_phi_j )
    Calculates if phi is necessarily true in all worlds accessible from w.
    """
    # A: [W, W] accessibility matrix
    # L_phi: [W] truth values of proposition phi in each world
    # Logic: If a world is accessible (A~1), L_phi must be high or the softmin collapses.
    expanded_L = L_phi.unsqueeze(0).expand(A.size(0), -1)
    return softmin((1.0 - A) + expanded_L)

# --- 2. THE MLNN ARCHITECTURE ---

class MLNN_Reasoner(nn.Module):
    def __init__(self, num_worlds):
        super().__init__()
        self.num_worlds = num_worlds

        # Accessibility Relation (A_theta): Using direct logits for stability in small worlds.
        # Initialized with a "Distrust Prior" (Section 3.3, 7.3)
        self.A_logits = nn.Parameter(torch.ones(num_worlds, num_worlds) * -2.0)

        # Proposition Truth Bounds [L, U]: Section 3.1 & 3.2
        # Initialize L < U to avoid immediate contradiction.
        self.L_p = nn.Parameter(torch.rand(num_worlds) * 0.3)
        self.U_p = nn.Parameter(0.7 + torch.rand(num_worlds) * 0.3)

    def get_A(self):
        """Pass logits through sigmoid to ensure weights in [0, 1] (Section 142, 182)."""
        return torch.sigmoid(self.A_logits)

    def forward(self):
        A = self.get_A()

        # Evaluation: Necessity of p (Box p)
        L_box_p = mlnn_box(A, self.L_p)

        # Contradiction Loss (L_contra): Section 3.4
        # Rule: Box p -> p. Contradiction occurs if L_box_p (premise) > U_p (conclusion).
        l_contra = torch.mean(torch.relu(L_box_p - self.U_p)**2)

        return A, l_contra

# --- 3. THE EXPERIMENT LOOP (TODO 1) ---

def run_reflexivity_sweep(lambda_t, num_worlds=10):
    model = MLNN_Reasoner(num_worlds)
    # Smaller learning rate to prevent structural collapse (Section 6.1)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Ground Truth: Directed Ring (Section 5.6)
    # i can see (i+1). This structure is NOT reflexive.
    ring = torch.zeros(num_worlds, num_worlds)
    for i in range(num_worlds):
        ring[i, (i + 1) % num_worlds] = 1.0

    for epoch in range(200):
        optimizer.zero_grad()
        A, l_contra = model()

        # Loss 1: Task Accuracy (Fitting the Ring)
        l_task = torch.mean((A - ring)**2)

        # Loss 2: Axiomatic Regularization (Equation 4 - Reflexivity)
        # Forces the diagonal A[i,i] toward 1.0.
        l_reflexive = torch.mean((1.0 - torch.diag(A))**2)

        # Total Loss (Equation 3 + 4)
        total_loss = l_task + (1.0 * l_contra) + (lambda_t * l_reflexive)

        total_loss.backward()
        optimizer.step()

        # Maintain valid bounds [0, 1] (Section 197)
        with torch.no_grad():
            model.L_p.clamp_(0, 1)
            model.U_p.clamp_(0, 1)

    # Calculate metrics for final state
    final_A = model.get_A().detach()
    epsilon_r = torch.mean(1.0 - torch.diag(final_A)).item()
    task_mse = torch.mean((final_A - ring)**2).item()

    return final_A.numpy(), epsilon_r, task_mse

# --- 4. EXECUTION & VISUALIZATION ---
lambdas = [0.0, 0.1, 0.5, 1.0, 5.0, 10.0]
plot_data = {'l': [], 'eps': [], 'mse': []}
final_matrices = []

print("Executing empirical analysis for TODO 1...")
for lt in lambdas:
    matrix, eps, mse = run_reflexivity_sweep(lt)
    plot_data['l'].append(lt)
    plot_data['eps'].append(eps)
    plot_data['mse'].append(mse)
    final_matrices.append(matrix)

# --- SAVE PLOTS TO INDIVIDUAL PDFS ---

# 1. Reflexivity Error Plot (A)
plt.figure(figsize=(6, 2))
plt.plot(lambdas, plot_data['eps'], 'b-o', markersize=4)
plt.title("Plot (a): Reflexivity Error ($\\epsilon_R$) vs $\\lambda_T$")
plt.xlabel("$\\lambda_T$")
plt.ylabel("$\\epsilon_R$")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('MLNN_AxiomT_Reflexivity_Error_Analysis.pdf')
plt.close()

print("Reflexivity Error ($\\epsilon_R$) vs $\\lambda_T$")
print(list(zip(lambdas, plot_data['eps'])))

# 2. Task Performance Plot (B)
plt.figure(figsize=(6, 2))
plt.plot(lambdas, plot_data['mse'], 'r-s', markersize=4)
plt.title("Plot (b): Structure MSE vs $\\lambda_T$")
plt.xlabel("$\\lambda_T$")
plt.ylabel("MSE")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('MLNN_AxiomT_Task_Performance_Analysis.pdf')
plt.close()

print("Structure MSE vs $\\lambda_T$")
print(list(zip(lambdas, plot_data['mse'])))

# 3. Heatmap Comparison Plot (C)
fig, axes = plt.subplots(1, 2, figsize=(6, 2))
axes[0].imshow(final_matrices[0], cmap='viridis', vmin=0, vmax=1)
axes[0].set_title("$\\lambda_T=0$ (Ring Focus)")
axes[1].imshow(final_matrices[-1], cmap='viridis', vmin=0, vmax=1)
axes[1].set_title("$\\lambda_T=10$ (Reflexive Focus)")
for ax in axes: ax.axis('off')
plt.tight_layout()
plt.savefig('MLNN_AxiomT_Structural_Heatmap_Comparison.pdf')
plt.close()

print("lambda_T=0$ (Ring Focus)")
print(final_matrices[0])
print("lambda_T=10$ (Reflexive Focus)")
print(final_matrices[-1])

Executing empirical analysis for TODO 1...
Reflexivity Error ($\epsilon_R$) vs $\lambda_T$
[(0.0, 0.9600136876106262), (0.1, 0.5553794503211975), (0.5, 0.4759384095668793), (1.0, 0.46787261962890625), (5.0, 0.4617440104484558), (10.0, 0.4609870910644531)]
Structure MSE vs $\lambda_T$
[(0.0, 0.022634398192167282), (0.1, 0.042240120470523834), (0.5, 0.04993817210197449), (1.0, 0.05078938975930214), (5.0, 0.05148319900035858), (10.0, 0.05154617130756378)]
lambda_T=0$ (Ring Focus)
[[0.04014131 0.5397581  0.04014131 0.04014131 0.04014131 0.04014131
  0.04014131 0.04014131 0.04014131 0.04014131]
 [0.03934409 0.03931469 0.5395105  0.04001489 0.03948966 0.03952682
  0.03995282 0.03976201 0.040154   0.03930203]
 [0.04014131 0.04014131 0.04014131 0.5397581  0.04014131 0.04014131
  0.04014131 0.04014131 0.04014131 0.04014131]
 [0.04014131 0.04014131 0.04014131 0.04014131 0.5397581  0.04014131
  0.04014131 0.04014131 0.04014131 0.04014131]
 [0.04014131 0.04014131 0.04014131 0.04014131 0.04014131 0

# Axiom $4$

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# --- 1. CORE MLNN OPERATORS (Differentiable Kripke Semantics) ---

def softmin(x, tau=0.1):
    """Equation 1 & Section 3.2.1: Sound lower bound on necessity."""
    return -tau * torch.log(torch.sum(torch.exp(-x / tau), dim=-1) + 1e-8)

def mlnn_box(A, L_phi):
    """Implements Necessity Neuron (Section 3.2.1)."""
    expanded_L = L_phi.unsqueeze(0).expand(A.size(0), -1)
    return softmin((1.0 - A) + expanded_L)

# --- 2. THE MLNN ARCHITECTURE ---

class MLNN_Reasoner(nn.Module):
    def __init__(self, num_worlds):
        super().__init__()
        self.num_worlds = num_worlds
        self.A_logits = nn.Parameter(torch.ones(num_worlds, num_worlds) * -2.0)
        self.L_p = nn.Parameter(torch.rand(num_worlds) * 0.3)
        self.U_p = nn.Parameter(0.7 + torch.rand(num_worlds) * 0.3)

    def get_A(self):
        return torch.sigmoid(self.A_logits)

    def forward(self):
        A = self.get_A()
        L_box_p = mlnn_box(A, self.L_p)
        # Contradiction Loss (L_contra): Section 3.4
        l_contra = torch.mean(torch.relu(L_box_p - self.U_p)**2)
        return A, l_contra

# --- 3. THE EXPERIMENT LOOP (TODO 1 - Adjusted for Axiom 4) ---

def run_transitivity_sweep(lambda_4, num_worlds=10):
    model = MLNN_Reasoner(num_worlds)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Ground Truth: Directed Ring (Section 5.6)
    ring = torch.zeros(num_worlds, num_worlds)
    for i in range(num_worlds):
        ring[i, (i + 1) % num_worlds] = 1.0

    for epoch in range(200):
        optimizer.zero_grad()
        A, l_contra = model()

        # Loss 1: Task Accuracy (Fitting the Ring)
        l_task = torch.mean((A - ring)**2)

        # Loss 2: Axiomatic Regularization (Equation 5 - Transitivity)
        # Enforces: A_direct >= A_path_length_2 [cite: 323, 325]
        A_sq = torch.matmul(A, A)
        l_transitive = torch.mean(torch.relu(A_sq - A)**2)

        # Total Loss (Equation 3 + 5)
        total_loss = l_task + (1.0 * l_contra) + (lambda_4 * l_transitive)

        total_loss.backward()
        optimizer.step()

        with torch.no_grad():
            model.L_p.clamp_(0, 1)
            model.U_p.clamp_(0, 1)

    # Calculate metrics
    final_A = model.get_A().detach()
    # Transitivity Error: Magnitude of path violations
    A_sq_final = torch.matmul(final_A, final_A)
    epsilon_4 = torch.mean(torch.relu(A_sq_final - final_A)).item()
    task_mse = torch.mean((final_A - ring)**2).item()

    return final_A.numpy(), epsilon_4, task_mse

# --- 4. EXECUTION & VISUALIZATION ---
lambdas = [0.0, 0.1, 0.5, 1.0, 5.0, 10.0]
plot_data = {'l': [], 'eps': [], 'mse': []}
final_matrices = []

print("Executing empirical analysis for TODO 1 (Axiom 4)...")
for l4 in lambdas:
    matrix, eps, mse = run_transitivity_sweep(l4)
    plot_data['l'].append(l4)
    plot_data['eps'].append(eps)
    plot_data['mse'].append(mse)
    final_matrices.append(matrix)

# 1. Transitivity Error Plot
plt.figure(figsize=(6, 3))
plt.plot(lambdas, plot_data['eps'], 'g-o', markersize=4)
plt.title("Plot (a): Transitivity Error ($\\epsilon_4$) vs $\\lambda_4$")
plt.xlabel("$\\lambda_4$")
plt.ylabel("$\\epsilon_4$")
plt.grid(True, alpha=0.3)
plt.savefig('MLNN_Axiom4_Transitivity_Error_Analysis.pdf')
plt.close()

print('Transitivity Error ($\\epsilon_4$) vs $\\lambda_4$')
print(list(zip(lambdas, plot_data['eps'])))

# 2. Task Performance Plot
plt.figure(figsize=(6, 3))
plt.plot(lambdas, plot_data['mse'], 'r-s', markersize=4)
plt.title("Plot (b): Structure MSE vs $\\lambda_4$")
plt.xlabel("$\\lambda_4$")
plt.ylabel("MSE")
plt.grid(True, alpha=0.3)
plt.savefig('MLNN_Axiom4_Transitivity_Task_Performance.pdf')
plt.close()

print('Structure MSE vs $\\lambda_4$')
print(list(zip(lambdas, plot_data['mse'])))

# 3. Heatmap Comparison
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(final_matrices[0], cmap='viridis', vmin=0, vmax=1)
axes[0].set_title("$\\lambda_4=0$ (Ring Graph)")
axes[1].imshow(final_matrices[-1], cmap='viridis', vmin=0, vmax=1)
axes[1].set_title("$\\lambda_4=10$ (Transitive Closure)")
for ax in axes: ax.axis('off')
plt.tight_layout()
plt.savefig('MLNN_Axiom4_Transitivity_Heatmap_Comparison.pdf')
plt.close()

print("lambda_4=0$ (Ring Graph)")
print(final_matrices[0])
print("lambda_4=10$ (Transitive Closure)")
print(final_matrices[-1])

Executing empirical analysis for TODO 1 (Axiom 4)...
Transitivity Error ($\epsilon_4$) vs $\lambda_4$
[(0.0, 0.03934779390692711), (0.1, 0.03895629569888115), (0.5, 0.03733643889427185), (1.0, 0.03614405542612076), (5.0, 0.02606678567826748), (10.0, 0.018386483192443848)]
Structure MSE vs $\lambda_4$
[(0.0, 0.022637639194726944), (0.1, 0.02294459566473961), (0.5, 0.024233411997556686), (1.0, 0.025784723460674286), (5.0, 0.034503377974033356), (10.0, 0.04063522443175316)]
lambda_4=0$ (Ring Graph)
[[0.04014131 0.5397581  0.04014131 0.04014131 0.04014131 0.04014131
  0.04014131 0.04014131 0.04014131 0.04014131]
 [0.03958998 0.03975794 0.5395108  0.0400685  0.03999669 0.03997527
  0.03959892 0.04003603 0.0399506  0.03957345]
 [0.0391896  0.03937645 0.03944352 0.539659   0.03983946 0.03979544
  0.03918738 0.03992074 0.03974523 0.04106038]
 [0.04014131 0.04014131 0.04014131 0.04014131 0.5397581  0.04014131
  0.04014131 0.04014131 0.04014131 0.04014131]
 [0.04014131 0.04014131 0.04014131 0.04

# Axiom $B$

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# --- 1. CORE MLNN OPERATORS ---

def softmin(x, tau=0.1):
    return -tau * torch.log(torch.sum(torch.exp(-x / tau), dim=-1) + 1e-8)

def mlnn_box(A, L_phi):
    expanded_L = L_phi.unsqueeze(0).expand(A.size(0), -1)
    return softmin((1.0 - A) + expanded_L)

# --- 2. THE MLNN ARCHITECTURE ---

class MLNN_Reasoner(nn.Module):
    def __init__(self, num_worlds):
        super().__init__()
        self.num_worlds = num_worlds
        self.A_logits = nn.Parameter(torch.ones(num_worlds, num_worlds) * -2.0)
        self.L_p = nn.Parameter(torch.rand(num_worlds) * 0.3)
        self.U_p = nn.Parameter(0.7 + torch.rand(num_worlds) * 0.3)

    def get_A(self):
        return torch.sigmoid(self.A_logits)

    def forward(self):
        A = self.get_A()
        L_box_p = mlnn_box(A, self.L_p)
        l_contra = torch.mean(torch.relu(L_box_p - self.U_p)**2)
        return A, l_contra

# --- 3. THE EXPERIMENT LOOP (Axiom B) ---

def run_symmetry_sweep(lambda_b, num_worlds=10):
    model = MLNN_Reasoner(num_worlds)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Ground Truth: Directed Ring
    ring = torch.zeros(num_worlds, num_worlds)
    for i in range(num_worlds):
        ring[i, (i + 1) % num_worlds] = 1.0

    for epoch in range(250):
        optimizer.zero_grad()
        A, l_contra = model()

        # Loss 1: Task Accuracy
        l_task = torch.mean((A - ring)**2)

        # Loss 2: Axiomatic Regularization (Equation 6 - Symmetry)
        # Penalizes A[i,j] != A[j,i]
        l_symmetric = torch.mean((A - A.transpose(0, 1))**2)

        # Total Loss
        total_loss = l_task + (1.0 * l_contra) + (lambda_b * l_symmetric)

        total_loss.backward()
        optimizer.step()

        with torch.no_grad():
            model.L_p.clamp_(0, 1)
            model.U_p.clamp_(0, 1)

    final_A = model.get_A().detach()
    # Symmetry Error: Difference between A and its transpose
    epsilon_b = torch.mean(torch.abs(final_A - final_A.transpose(0, 1))).item()
    task_mse = torch.mean((final_A - ring)**2).item()

    return final_A.numpy(), epsilon_b, task_mse

# --- 4. EXECUTION ---
lambdas = [0.0, 0.1, 0.5, 1.0, 5.0, 10.0]
plot_data = {'l': [], 'eps': [], 'mse': []}
final_matrices = []

print("Executing Symmetry Analysis (Axiom B)...")
for lb in lambdas:
    matrix, eps, mse = run_symmetry_sweep(lb)
    plot_data['l'].append(lb)
    plot_data['eps'].append(eps)
    plot_data['mse'].append(mse)
    final_matrices.append(matrix)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(final_matrices[0], cmap='magma', vmin=0, vmax=1)
axes[0].set_title("$\\lambda_B=0$ (Directed Ring)")
axes[1].imshow(final_matrices[-1], cmap='magma', vmin=0, vmax=1)
axes[1].set_title("$\\lambda_B=10$ (Symmetric Graph)")
for ax in axes: ax.axis('off')
plt.tight_layout()
plt.savefig('MLNN_AxiomB_Symmetry_Heatmap_Comparison.pdf')
plt.close()


print("lambda_B=0$ (Directed Ring)")
print(final_matrices[0])
print("lambda_B=10$ (Symmetric Graph)")
print(final_matrices[-1])

Executing Symmetry Analysis (Axiom B)...
lambda_B=0$ (Directed Ring)
[[0.0347987  0.6378795  0.0347987  0.0347987  0.0347987  0.0347987
  0.0347987  0.0347987  0.0347987  0.0347987 ]
 [0.0347987  0.0347987  0.6378795  0.0347987  0.0347987  0.0347987
  0.0347987  0.0347987  0.0347987  0.0347987 ]
 [0.0347987  0.0347987  0.0347987  0.6378795  0.0347987  0.0347987
  0.0347987  0.0347987  0.0347987  0.0347987 ]
 [0.0347987  0.0347987  0.0347987  0.0347987  0.6378795  0.0347987
  0.0347987  0.0347987  0.0347987  0.0347987 ]
 [0.03475046 0.03471795 0.03442907 0.03452768 0.03467333 0.63785034
  0.03438036 0.03451167 0.03470337 0.03457715]
 [0.0347987  0.0347987  0.0347987  0.0347987  0.0347987  0.0347987
  0.6378795  0.0347987  0.0347987  0.0347987 ]
 [0.0347987  0.0347987  0.0347987  0.0347987  0.0347987  0.0347987
  0.0347987  0.6378795  0.0347987  0.0347987 ]
 [0.0347987  0.0347987  0.0347987  0.0347987  0.0347987  0.0347987
  0.0347987  0.0347987  0.6378795  0.0347987 ]
 [0.0347987  0.034